In [ ]:
import json
import os
import pandas as pd
from research.utils.data_access_utils import S3AccessUtils
from report_generation.report_generator import generate_ts_data, SamplingFilter
from research.utils.datetime_utils import add_days
from report_generation.report_generator import gen_pm_base
from population_metrics.smart_metrics import generate_smart_avg_weight, generate_smart_individual_values, ValidationError
from filter_optimization.filter_optimization_task import _add_date_hour_columns
from research.weight_estimation.keypoint_utils.optics import pixel2world
import numpy as np

pd.set_option('display.max_rows', 500)

In [ ]:
s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))

In [ ]:
cohort_names = [
    'seglberget_pen_id_66_2020-05-13_2020-06-13',
    'bolaks_pen_id_88_2020-02-28_2020-03-10',
    'langoy_pen_id_108_2020-05-07_2020-05-17',
    'tittelsnes_pen_id_37_2020-06-10_2020-06-24',
    'aplavika_pen_id_95_2020-07-10_2020-07-26',
#     'kjeppevikholmen_pen_id_5_2019-06-18_2019-07-02',
    'silda_pen_id_86_2020-07-02_2020-07-19',
    'vikane_pen_id_60_2020-08-10_2020-08-30',
    'eldviktaren_pen_id_164_2020-09-21_2020-10-08',
#     'habranden_pen_id_100_2020-08-10_2020-08-31',
    'varholmen_pen_id_131_2020-08-15_2020-08-30',
    'dale_pen_id_143_2020-10-07_2020-10-21',
    'djubawik_pen_id_153_2020-11-10_2020-11-26',
    'leivsethamran_pen_id_165_2020-10-18_2020-11-13',
    'movikodden_pen_id_114_2020-11-03_2020-11-25',
    'movikodden_pen_id_167_2020-10-13_2020-10-30',
    'slapoya_pen_id_116_2020-10-18_2020-11-08',
    'varholmen_pen_id_131_2020-08-15_2020-08-30',
    'varholmen_pen_id_151_2020-10-02_2020-10-17',
    'varholmen_pen_id_186_2020-10-18_2020-11-02'
]

In [ ]:
batch_name = 'test'

ROOT_DIR = '/root/data/alok/biomass_estimation/playground'
dfs, gt_metadatas = {}, {}
for cohort_name in cohort_names:
    s3_dir = os.path.join(
        'https://aquabyte-images-adhoc.s3-eu-west-1.amazonaws.com/alok/production_datasets',
        cohort_name
    )

    ground_truth_metadata_url = os.path.join(s3_dir, 'ground_truth_metadata.json')
    ground_truth_key_base = os.path.join(batch_name, cohort_name, 'ground_truth_metadata.json')
#     ground_truth_metadata_url = os.path.join(s3_dir, 'ground_truth_metadata_validated.json')
#     ground_truth_key_base = os.path.join(batch_name, cohort_name, 'ground_truth_metadata_validated.json')
    ground_truth_f = os.path.join(ROOT_DIR, ground_truth_key_base)
    print(ground_truth_metadata_url)
    s3.download_from_url(ground_truth_metadata_url, custom_location=ground_truth_f)
    gt_metadata = json.load(open(ground_truth_f))
    gt_metadatas[cohort_name] = gt_metadata
    
    data_url = os.path.join(s3_dir, 'annotation_dataset.csv')
    data_f, _, _= s3.download_from_url(data_url)
    df = pd.read_csv(data_f)
    df = _add_date_hour_columns(df)
    dfs[cohort_name] = df
    
    

<h1> Generate old / new model weights </h1>

In [ ]:

    
"""This module contains utility helper functions for the WeightEstimator class."""

from collections import namedtuple
from typing import Dict, List, Tuple
import numpy as np
import torch
from research.weight_estimation.keypoint_utils import body_parts


CameraMetadata = namedtuple('CameraMetadata',
                            ['focal_length', 'focal_length_pixel', 'baseline_m',
                             'pixel_count_width', 'pixel_count_height', 'image_sensor_width',
                             'image_sensor_height'])


def get_left_right_keypoint_arrs(annotation: Dict[str, List[Dict]]) -> Tuple:
    """Gets numpy array of left and right keypoints given input keypoint annotation.
    Args:
        annotation: dict with keys 'leftCrop' and 'rightCrop'. Values are lists where each element
        is a dict with keys 'keypointType', 'xCrop' (num pixels from crop left edge),
        'yCrop' (num pixels from crop top edge), 'xFrame' (num pixels from full frame left edge),
        and 'yFrame' (num pixels from full frame top edge).
    Returns:
        X_left: numpy array containing left crop (xFrame, yFrame) for each key-point ordered
        alphabetically.
        X_right: same as above, but for right crop.
    """

    left_keypoints, right_keypoints = {}, {}
    for item in annotation['leftCrop']:
        body_part = item['keypointType']
        left_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    for item in annotation['rightCrop']:
        body_part = item['keypointType']
        right_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    left_keypoint_arr, right_keypoint_arr = [], []
    for body_part in body_parts.core_body_parts:
        left_keypoint_arr.append(left_keypoints[body_part])
        right_keypoint_arr.append(right_keypoints[body_part])

    X_left = np.array(left_keypoint_arr)
    X_right = np.array(right_keypoint_arr)
    return X_left, X_right


def normalize_left_right_keypoint_arrs(X_left: np.ndarray, X_right: np.ndarray) -> Tuple:
    """Normalizes input left and right key-point arrays. The normalization involves (1) 2D
    translation of all keypoints such that they are centered, (2) rotation of the 2D coordiantes
    about the center such that the line passing through UPPER_LIP and fish center is horizontal.
    """

    # translate key-points, perform reflection if necessary
    upper_lip_idx = body_parts.core_body_parts.index(body_parts.UPPER_LIP)
    tail_notch_idx = body_parts.core_body_parts.index(body_parts.TAIL_NOTCH)
    if X_left[upper_lip_idx, 0] > X_left[tail_notch_idx, 0]:
        X_center = 0.5 * (np.max(X_left, axis=0) + np.min(X_left, axis=0))
        X_left_centered = X_left - X_center
        X_right_centered = X_right - X_center
    else:
        X_center = 0.5 * (np.max(X_right, axis=0) + np.min(X_right, axis=0))
        X_left_centered = X_right - X_center
        X_right_centered = X_left - X_center
        X_left_centered[:, 0] = -X_left_centered[:, 0]
        X_right_centered[:, 0] = -X_right_centered[:, 0]

    # rotate key-points
    upper_lip_x, upper_lip_y = tuple(X_left_centered[upper_lip_idx])
    theta = np.arctan(upper_lip_y / upper_lip_x)
    R = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])

    D = X_left_centered - X_right_centered
    X_left_rot = np.dot(X_left_centered, R)
    X_right_rot = X_left_rot - D
    return X_left_rot, X_right_rot


def convert_to_world_point_arr(X_left: np.ndarray, X_right: np.ndarray,
                               camera_metadata: CameraMetadata) -> np.ndarray:
    """Converts input left and right normalized keypoint arrays into world coordinate array."""

    y_world = camera_metadata.focal_length_pixel * camera_metadata.baseline_m / \
              (X_left[:, 0] - X_right[:, 0])

    # Note: the lines commented out below are technically the correct formula for conversion
    # x_world = X_left[:, 0] * y_world / camera_metadata.focal_length_pixel
    # z_world = -X_left[:, 1] * y_world / camera_metadata.focal_length_pixel
    x_world = ((X_left[:, 0] * camera_metadata.image_sensor_width / camera_metadata.pixel_count_width) * y_world) / (camera_metadata.focal_length)
    z_world = (-(X_left[:, 1] * camera_metadata.image_sensor_height / camera_metadata.pixel_count_height) * y_world) / (camera_metadata.focal_length)
    X_world = np.vstack([x_world, y_world, z_world]).T
    return X_world


def stabilize_keypoints(X: np.ndarray) -> np.ndarray:
    """Transforms world coordinate array so that neural network inputs are stabilized"""
    X_new = np.zeros(X.shape)
    X_new[:, 0] = 0.5 * X[:, 0] / X[:, 1]
    X_new[:, 1] = 0.5 * X[:, 2] / X[:, 1]
    X_new[:, 2] = 0.05 / X[:, 1]
    return X_new


def convert_to_nn_input(annotation: Dict[str, List[Dict]], camera_metadata: CameraMetadata) \
        -> torch.Tensor:
    """Convrts input keypoint annotation and camera metadata into neural network tensor input."""
    X_left, X_right = get_left_right_keypoint_arrs(annotation)
    X_left_norm, X_right_norm = normalize_left_right_keypoint_arrs(X_left, X_right)
    X_world = convert_to_world_point_arr(X_left_norm, X_right_norm, camera_metadata)
    X = stabilize_keypoints(X_world)
    nn_input = torch.from_numpy(np.array([X])).float()
    return nn_input


"""
This module contains the WeightEstimator class for estimating fish weight (g), length (mm), and
k-factor given input keypoint coordinates and camera metadata.
"""

from typing import Dict, Tuple
import torch
from torch import nn


class Network(nn.Module):
    """Network class defines neural-network architecture for both weight and k-factor estimation
    (currently both neural networks share identical architecture)."""

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(24, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        """Run inference on input keypoint tensor."""
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.output(x)
        return x


class WeightEstimator:
    """WeightEstimator class is used to predict fish weight, k-factor, and length
    given input keypoint annotations and camera metadata."""

    def __init__(self, weight_model_f: str, kf_model_f: str) -> None:
        """Initializes class with input weight and k-factor neural-networks."""
        self.weight_model = Network()
        self.weight_model.load_state_dict(torch.load(weight_model_f))
        self.weight_model.eval()

        self.kf_model = Network()
        self.kf_model.load_state_dict(torch.load(kf_model_f))
        self.kf_model.eval()

    @staticmethod
    def _get_model_input(annotation: Dict, camera_metadata: CameraMetadata) -> torch.Tensor:
        """Generates neural-network input tensor given annotation and camera_metadata."""
        X = convert_to_nn_input(annotation, camera_metadata)
        return X

    def predict_weight(self, annotation: Dict, camera_metadata: CameraMetadata) -> float:
        """Generates weight prediction given input annotation and camera metadata."""
        X = self._get_model_input(annotation, camera_metadata)
        weight = 1e4 * self.weight_model(X).item()
        return weight

    def predict_kf(self, annotation: Dict, camera_metadata: CameraMetadata) -> float:
        """Generates k-factor prediction gievn input annotation and camera metadata."""
        X = self._get_model_input(annotation, camera_metadata)
        kf = self.kf_model(X).item()
        return kf

    def predict(self, annotation: Dict, camera_metadata: CameraMetadata) -> Tuple:
        """Generates weight, k-factor, and length predictions given input annotation and camera
        metadata."""
        weight = self.predict_weight(annotation, camera_metadata)
        kf = self.predict_kf(annotation, camera_metadata)
        if weight * kf > 0:
            length = (1e5 * weight / kf) ** (1.0 / 3)
        else:
            length = 0
        return weight, length, kf

In [ ]:
# weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb')
weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/playground/nn_epoch_798_v2.pb')
kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/playground/kf_predictor_v2.pb')

weight_estimator = WeightEstimator(weight_model_f, kf_model_f)


for k, rdf in dfs.items():
    print(k)
    weights = []
    count = 0
    for idx, row in rdf.iterrows():
        if count % 100 == 0:
            print('Percentage completion: {}%'.format(round(100 * count / rdf.shape[0], 2)))
            print(count)
        count += 1
        annotation = json.loads(row.annotation.replace("'", '"'))
        if not annotation:
            weights.append(None)
            continue
        camera_metadata = json.loads(row.camera_metadata.replace("'", '"'))
        if not camera_metadata:
            camera_metadata = json.loads(rdf.camera_metadata.iloc[0].replace("'", '"'))

        camera_metadata_obj = CameraMetadata(
            focal_length=camera_metadata['focalLength'],
            focal_length_pixel=camera_metadata['focalLengthPixel'],
            baseline_m=camera_metadata['baseline'],
            pixel_count_width=camera_metadata['pixelCountWidth'],
            pixel_count_height=camera_metadata['pixelCountHeight'],
            image_sensor_width=camera_metadata['imageSensorWidth'],
            image_sensor_height=camera_metadata['imageSensorHeight']
        )

        weight, length, kf = weight_estimator.predict(annotation, camera_metadata_obj)
        weights.append(weight)
    rdf['weight_v1'] = weights


In [ ]:
weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb')
# weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/playground/nn_epoch_798_v2.pb')
kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/playground/kf_predictor_v2.pb')

weight_estimator = WeightEstimator(weight_model_f, kf_model_f)


for k, rdf in dfs.items():
    print(k)
    weights = []
    count = 0
    for idx, row in rdf.iterrows():
        if count % 100 == 0:
            print('Percentage completion: {}%'.format(round(100 * count / rdf.shape[0], 2)))
            print(count)
        count += 1
        annotation = json.loads(row.annotation.replace("'", '"'))
        if not annotation:
            weights.append(None)
            continue
        camera_metadata = json.loads(row.camera_metadata.replace("'", '"'))
        if not camera_metadata:
            camera_metadata = json.loads(rdf.camera_metadata.iloc[0].replace("'", '"'))

        camera_metadata_obj = CameraMetadata(
            focal_length=camera_metadata['focalLength'],
            focal_length_pixel=camera_metadata['focalLengthPixel'],
            baseline_m=camera_metadata['baseline'],
            pixel_count_width=camera_metadata['pixelCountWidth'],
            pixel_count_height=camera_metadata['pixelCountHeight'],
            image_sensor_width=camera_metadata['imageSensorWidth'],
            image_sensor_height=camera_metadata['imageSensorHeight']
        )

        weight, length, kf = weight_estimator.predict(annotation, camera_metadata_obj)
        weights.append(weight)
    rdf['weight_v2'] = weights


In [ ]:
weight_model_f, _, _ = s3.download_from_url('https://aquabyte-images-adhoc.s3-eu-west-1.amazonaws.com/alok/new_pytorch_model.pb')
# weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/playground/nn_epoch_798_v2.pb')
kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/playground/kf_predictor_v2.pb')

weight_estimator = WeightEstimator(weight_model_f, kf_model_f)


for k, rdf in dfs.items():
    print(k)
    weights = []
    count = 0
    for idx, row in rdf.iterrows():
        if count % 100 == 0:
            print('Percentage completion: {}%'.format(round(100 * count / rdf.shape[0], 2)))
            print(count)
        count += 1
        annotation = json.loads(row.annotation.replace("'", '"'))
        if not annotation:
            weights.append(None)
            continue
        camera_metadata = json.loads(row.camera_metadata.replace("'", '"'))
        if not camera_metadata:
            camera_metadata = json.loads(rdf.camera_metadata.iloc[0].replace("'", '"'))

        camera_metadata_obj = CameraMetadata(
            focal_length=camera_metadata['focalLength'],
            focal_length_pixel=camera_metadata['focalLengthPixel'],
            baseline_m=camera_metadata['baseline'],
            pixel_count_width=camera_metadata['pixelCountWidth'],
            pixel_count_height=camera_metadata['pixelCountHeight'],
            image_sensor_width=camera_metadata['imageSensorWidth'],
            image_sensor_height=camera_metadata['imageSensorHeight']
        )

        weight, length, kf = weight_estimator.predict(annotation, camera_metadata_obj)
        weights.append(weight)
    rdf['weight_v3'] = weights


<h1> Generate average weight accuracy with old model </h1>

In [ ]:
def generate_raw_individual_values(pm_base, gt_metadata, start_hour, end_hour, apply_growth_rate, max_day_diff, days_post_feeding, final_days_post_feeding):
    last_feeding_date = gt_metadata['last_feeding_date']
    date = add_days(last_feeding_date, days_post_feeding)
    weights, _ = generate_smart_individual_values(pm_base, date, max_day_diff, True, apply_growth_rate, 0.9)
    return weights


def generate_average_weight_accuracy(weights, gt_metadata, loss_factor):
    avg_weight_prediction = np.mean(weights)
    gutted_weight_prediction = avg_weight_prediction * (1.0 - loss_factor)
    gt_weight = gt_metadata['gutted_average_weight']
    avg_weight_err = (gutted_weight_prediction - gt_weight) / gt_weight
    return avg_weight_err

def generate_distribution_accuracy(weights, gt_metadata, loss_factor):
    gutted_weights = weights * (1.0 - loss_factor)
    gutted_weight_distribution = gt_metadata['gutted_weight_distribution']
    
    if gutted_weight_distribution is None:
        return []
    
    count_distribution_errors = []
    
    for bucket in gutted_weight_distribution:
        lower_bound, upper_bound = bucket.split('-')
        pct = gutted_weight_distribution[bucket]
        mask = (gutted_weights >= float(lower_bound) * 1000) & (gutted_weights < float(upper_bound) * 1000)

        pct = np.sum(mask) / len(mask)
        gt_pct = gutted_weight_distribution[bucket] / 100
        
        count_distribution_errors.append(pct - gt_pct)
        
    return count_distribution_errors



In [ ]:
start_hours = [7]
end_hours = [15]
apply_growth_rate = True
max_day_diff = 3
days_post_feeding = 1
final_days_post_feeding = 3
loss_factors = [0.14, 0.16] # need to determine the right values here

cohort_name_col = []
start_hour_col = []
end_hour_col = []
loss_factor_col = []
avg_weight_error_col = []
count_distribution_error_col = []

for loss_factor in loss_factors:
    avg_weight_error_col.append([])
    count_distribution_error_col.append([])

for cohort_name in sorted(list(dfs.keys())):
    gt_metadata = gt_metadatas[cohort_name]
    for start_hour in start_hours:
        for end_hour in end_hours:
            sampling_filter = SamplingFilter(
                start_hour=start_hour,
                end_hour=end_hour,
                kf_cutoff=0.0,
                akpd_score_cutoff=0.95
            )
            df = dfs[cohort_name]
            df['estimated_weight_g'] = df['weight_v1']
            final_date_post_feeding = add_days(gt_metadata['last_feeding_date'], final_days_post_feeding)
            tdf = df[df.date <= final_date_post_feeding]
            pm_base = gen_pm_base(tdf, sampling_filter)
            
            try:
                weights = generate_raw_individual_values(pm_base, gt_metadata, start_hour, end_hour, apply_growth_rate, max_day_diff, days_post_feeding, final_days_post_feeding)
            except ValidationError as err:
                continue
            
            cohort_name_col.append(cohort_name)
            start_hour_col.append(start_hour)
            end_hour_col.append(end_hour)
            loss_factor_col.append(loss_factor)
                   
            for index, loss_factor in enumerate(loss_factors):
                avg_weight_err = generate_average_weight_accuracy(weights, gt_metadata, loss_factor)
                avg_weight_error_col[index].append(avg_weight_err)
                
                count_distribution_errors = generate_distribution_accuracy(weights, gt_metadata, loss_factor)
                count_distribution_error_col[index].append(count_distribution_errors)


In [ ]:
columns = {
    'cohort_name': cohort_name_col,
    'start_hour_col': start_hour_col,
    'end_hour_col': end_hour_col
}

for index, loss_factor in enumerate(loss_factors):
    col_name = 'avg_weight_error_%0.2f' % (loss_factor,)
    col_abs_name = 'avg_weight_error_abs_%0.2f' % (loss_factor,)
    columns[col_name] = avg_weight_error_col[index]
    columns[col_abs_name] = np.abs(avg_weight_error_col[index])
    
    col_abs_name = 'avg_count_dist_error_abs_%0.2f' % (loss_factor,)
    columns[col_abs_name] = [np.mean(np.abs(l)) for l in count_distribution_error_col[index]]

tdf = pd.DataFrame(columns)

In [ ]:
tdf

<h1> Generate average weight accuracy with new model </h1>

In [ ]:
start_hours = [7]
end_hours = [15]
apply_growth_rate = True
max_day_diff = 3
days_post_feeding = 1
final_days_post_feeding = 3
loss_factors = [0.14, 0.16] # need to determine the right values here

cohort_name_col = []
start_hour_col = []
end_hour_col = []
loss_factor_col = []
avg_weight_error_col = []
count_distribution_error_col = []

for loss_factor in loss_factors:
    avg_weight_error_col.append([])
    count_distribution_error_col.append([])

for cohort_name in sorted(list(dfs.keys())):
    gt_metadata = gt_metadatas[cohort_name]
    for start_hour in start_hours:
        for end_hour in end_hours:
            sampling_filter = SamplingFilter(
                start_hour=start_hour,
                end_hour=end_hour,
                kf_cutoff=0.0,
                akpd_score_cutoff=0.01
            )
            df = dfs[cohort_name]
            df['estimated_weight_g'] = df['weight_v2']
            final_date_post_feeding = add_days(gt_metadata['last_feeding_date'], final_days_post_feeding)
            tdf = df[df.date <= final_date_post_feeding]
            pm_base = gen_pm_base(tdf, sampling_filter)
            
            try:
                weights = generate_raw_individual_values(pm_base, gt_metadata, start_hour, end_hour, apply_growth_rate, max_day_diff, days_post_feeding, final_days_post_feeding)
            except ValidationError as err:
                continue
            
            cohort_name_col.append(cohort_name)
            start_hour_col.append(start_hour)
            end_hour_col.append(end_hour)
            loss_factor_col.append(loss_factor)
                   
            for index, loss_factor in enumerate(loss_factors):
                avg_weight_err = generate_average_weight_accuracy(weights, gt_metadata, loss_factor)
                avg_weight_error_col[index].append(avg_weight_err)
                
                count_distribution_errors = generate_distribution_accuracy(weights, gt_metadata, loss_factor)
                count_distribution_error_col[index].append(count_distribution_errors)


In [ ]:
columns = {
    'cohort_name': cohort_name_col,
    'start_hour_col': start_hour_col,
    'end_hour_col': end_hour_col
}

for index, loss_factor in enumerate(loss_factors):
    col_name = 'avg_weight_error_%0.2f' % (loss_factor,)
    col_abs_name = 'avg_weight_error_abs_%0.2f' % (loss_factor,)
    columns[col_name] = avg_weight_error_col[index]
    columns[col_abs_name] = np.abs(avg_weight_error_col[index])
    
    col_abs_name = 'avg_count_dist_error_abs_%0.2f' % (loss_factor,)
    columns[col_abs_name] = [np.mean(np.abs(l)) for l in count_distribution_error_col[index]]

tdf = pd.DataFrame(columns)
tdf

In [ ]:
start_hours = [7]
end_hours = [15]
apply_growth_rate = True
max_day_diff = 3
days_post_feeding = 1
final_days_post_feeding = 3
loss_factors = [0.14, 0.16] # need to determine the right values here

cohort_name_col = []
start_hour_col = []
end_hour_col = []
loss_factor_col = []
avg_weight_error_col = []
count_distribution_error_col = []
gt_weight_col = []

for loss_factor in loss_factors:
    avg_weight_error_col.append([])
    count_distribution_error_col.append([])

for cohort_name in sorted(list(dfs.keys())):
    gt_metadata = gt_metadatas[cohort_name]
    for start_hour in start_hours:
        for end_hour in end_hours:
            sampling_filter = SamplingFilter(
                start_hour=start_hour,
                end_hour=end_hour,
                kf_cutoff=0.0,
                akpd_score_cutoff=0.01
            )
            df = dfs[cohort_name]
            df['estimated_weight_g'] = df['weight_v1']
            final_date_post_feeding = add_days(gt_metadata['last_feeding_date'], final_days_post_feeding)
            tdf = df[df.date <= final_date_post_feeding]
            pm_base = gen_pm_base(tdf, sampling_filter)
            
            try:
                weights = generate_raw_individual_values(pm_base, gt_metadata, start_hour, end_hour, apply_growth_rate, max_day_diff, days_post_feeding, final_days_post_feeding)
            except ValidationError as err:
                continue
            
            cohort_name_col.append(cohort_name)
            start_hour_col.append(start_hour)
            end_hour_col.append(end_hour)
            loss_factor_col.append(loss_factor)
            gt_weight_col.append(gt_metadata['gutted_average_weight'])
                   
            for index, loss_factor in enumerate(loss_factors):
                avg_weight_err = generate_average_weight_accuracy(weights, gt_metadata, loss_factor)
                avg_weight_error_col[index].append(avg_weight_err)
                
                count_distribution_errors = generate_distribution_accuracy(weights, gt_metadata, loss_factor)
                count_distribution_error_col[index].append(count_distribution_errors)


In [ ]:
columns = {
    'cohort_name': cohort_name_col,
    'start_hour_col': start_hour_col,
    'end_hour_col': end_hour_col,
    'gt_weight': gt_weight_col
}

for index, loss_factor in enumerate(loss_factors):
    col_name = 'avg_weight_error_%0.2f' % (loss_factor,)
    col_abs_name = 'avg_weight_error_abs_%0.2f' % (loss_factor,)
    columns[col_name] = avg_weight_error_col[index]
    columns[col_abs_name] = np.abs(avg_weight_error_col[index])
    
    col_abs_name = 'avg_count_dist_error_abs_%0.2f' % (loss_factor,)
    columns[col_abs_name] = [np.mean(np.abs(l)) for l in count_distribution_error_col[index]]

tdf = pd.DataFrame(columns)
tdf

In [ ]:
plt.scatter(tdf.gt_weight, tdf['avg_weight_error_0.14'])
plt.show()

In [ ]:
tdf.loc[(tdf.gt_weight < 5000),'avg_weight_error_0.14'] - 0.014

In [ ]:
]

In [ ]:
(tdf['avg_weight_error_0.14'] - 0.03)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
kdf = dfs['tittelsnes_pen_id_37_2020-06-10_2020-06-24']

In [ ]:
mask = kdf.weight_v1 > 5000

plt.figure(figsize=(20, 10))
plt.hist(kdf[mask].weight_v1 - kdf[mask].weight_v2, bins=200)
plt.grid()
plt.show()

In [ ]:
buckets = np.arange(0, 10000, 1000)
for low, high in zip(buckets, buckets[1:]):
    mask = (kdf.weight_v1 > low) & (kdf.weight_v1 < high) & (kdf.akpd_score > 0.99)
    pct_diff = (kdf[mask].weight_v2.mean() - kdf[mask].weight_v1.mean()) / kdf[mask].weight_v1.mean()
    print(low, high, pct_diff)

In [ ]:
index = 0
loss_factor = loss_factors[index]
col_abs_name = 'avg_weight_error_abs_%0.2f' % (loss_factor,)
error = tdf[col_abs_name]

print('Loss factor', loss_factor)

print()

print('Average Weight Error')
print('Avg %0.1f' % (np.mean(error) * 100, ))
print('90th Pct %0.1f' % (np.percentile(error, 90) * 100, ))
print('Max %0.1f' % (np.max(error) * 100, ))

print()

dist_errors = [item for sublist in count_distribution_error_col[index] for item in sublist]

print('Count Distribution Error')
print('Avg %0.1f' % (np.mean(np.abs(dist_errors)) * 100, ))
print('90th Pct %0.1f' % (np.percentile(np.abs(dist_errors), 90) * 100, ))
print('Max %0.1f' % (np.max(np.abs(dist_errors)) * 100, ))

In [ ]:
# for cohort_name in cohort_names:
#     mask = tdf.cohort_name == cohort_name
#     print(tdf[mask].sort_values('avg_weight_error_abs', ascending=True).head(10))

In [ ]:
gt_metadatas['vikane_pen_id_60_2020-08-05_2020-08-30']

In [ ]:
tdf.cohort_name.unique()

In [ ]:
mask = (tdf.cohort_name == 'tittelsnes_pen_id_37_2020-05-23_2020-06-24') & (tdf.days_post_feeding == 1) & (tdf.final_days_post_feeding == 1) & (tdf.max_day_diff == 3) & (tdf.loss_factor == 0.17)
tdf[mask].sort_values('avg_weight_error_abs')



In [ ]:
mask = (tdf.start_hour_col == 6) & (tdf.days_post_feeding == 1) & (tdf.final_days_post_feeding == 1) & (tdf.max_day_diff == 3)
tdf[mask].avg_weight_error_abs.median()



In [ ]:
mask = (tdf.start_hour_col == 7) & (tdf.days_post_feeding == 1) & (tdf.final_days_post_feeding == 1) & (tdf.max_day_diff == 3)
tdf[mask].avg_weight_error_abs.median()



In [ ]:
cohort_name_col = []
start_hour_col = []
end_hour_col = []
apply_growth_rate_col = []
max_day_diff_col = []
days_post_feeding_col = []
final_days_post_feeding_col = []
loss_factor_col = []
std_avg_weight_error_col = []
abs_avg_weight_error_col = []
mean_avg_weight_error_col = []

for start_hour in start_hours:
    for end_hour in end_hours:
        for apply_growth_rate in apply_growth_rate_list:
            for max_day_diff in max_day_diff_list:
                for days_post_feeding in days_post_feeding_list:
                    for final_days_post_feeding in final_days_post_feeding_list:
                        for loss_factor in loss_factors:
                            mask = (tdf.start_hour_col == start_hour) & \
                            (tdf.end_hour_col == end_hour) & \
                            (tdf.apply_growth_rate == apply_growth_rate) & \
                            (tdf.max_day_diff == max_day_diff) & \
                            (tdf.days_post_feeding == days_post_feeding) & \
                            (tdf.final_days_post_feeding == final_days_post_feeding) & \
                            (tdf.loss_factor == loss_factor)
                            
                            start_hour_col.append(start_hour)
                            end_hour_col.append(end_hour)
                            apply_growth_rate_col.append(apply_growth_rate)
                            max_day_diff_col.append(max_day_diff)
                            days_post_feeding_col.append(days_post_feeding)
                            final_days_post_feeding_col.append(final_days_post_feeding)
                            loss_factor_col.append(loss_factor)
                            std_avg_weight_error_col.append(tdf[mask].avg_weight_error.std())
                            abs_avg_weight_error_col.append(tdf[mask].avg_weight_error_abs.mean())
                            mean_avg_weight_error_col.append(tdf[mask].avg_weight_error.mean())

In [ ]:
rdf = pd.DataFrame({
    'start_hour_col': start_hour_col,
    'end_hour_col': end_hour_col,
    'apply_growth_rate': apply_growth_rate_col,
    'max_day_diff': max_day_diff_col,
    'days_post_feeding': days_post_feeding_col,
    'final_days_post_feeding': final_days_post_feeding_col,
    'loss_factor': loss_factor_col,
    'abs_avg_weight_error': abs_avg_weight_error_col,
    'std_avg_weight_error': std_avg_weight_error_col,
    'mean_avg_weight_error': mean_avg_weight_error_col,
})



In [ ]:
rdf

In [ ]:
mask = (rdf.loss_factor == 0.16)
rdf[mask].sort_values('abs_avg_weight_error')

In [ ]:
tdf.to_csv('/root/data/alok/biomass_estimation/playground/smart_average_param_grid_search.csv')

In [ ]:
tdf[(tdf.cohort_name == 'bolaks_pen_id_88_2020-02-10_2020-03-10')].sort_values('avg_weight_error_abs')



In [ ]:
# generate Vikane average weight and distribution error - explore basic parameters

ground_truth_metadata = json.load(open(ground_truth_f))
day_after_feeding_stop = add_days(ground_truth_metadata['last_feeding_date'], 1)
start_date, end_date = add_days(day_after_feeding_stop, -2), add_days(day_after_feeding_stop, -1)
tdf = df[(df.date >= start_date) & (df.date <= end_date)].copy(deep=True)

sampling_filter = SamplingFilter(
    start_hour=7,
    end_hour=15,
    akpd_score_cutoff=0.95,
    kf_cutoff=0.0
)
pm_base = gen_pm_base(tdf, sampling_filter)
weights, _ = generate_smart_individual_values(pm_base, day_after_feeding_stop, 3, True, True, 0.9)


In [ ]:
np.mean(weights)